# github doesn't support dynamic JS...

    > to view the plots go here here...
        - http://nbviewer.jupyter.org/github/FelicianoAnthony/Python_projects/blob/master/web-scraping/br_scraping_walkthrough.ipynb

In [1]:
import sys
sys.path.append('/path/to/baseball_analysis.py')

In [2]:
import baseball_analysis as table_scraper
table_scraper = table_scraper.html_table_scraper()

Loading BokehJS ...

# EXAMPLE 
    > scraped table 

In [3]:
df = table_scraper.scrape_batting_tables('https://www.baseball-reference.com/teams/NYM/2016.shtml')
# filter by all players under 25 who aren't pitchers and sort by home runs 
df.ix[(df['AGE'] < 25 ) & (df['POS'] != 'P')].sort_values('HR', ascending=False)

,,,RK,POS,AGE,G,PA,AB,R,H,2B,3B,...,OBP,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB
NAME,TEAM_NAME,YEAR,,,,,,,,,,,,,,,,,,,,,
Wilmer Flores,NYM,2016,9,CI,24,103,335,307,38,82,14,0,...,0.319,0.469,0.788,110,144,9,2,0,3,0
Michael Conforto,NYM,2016,6,LF,23,109,348,304,38,67,21,1,...,0.310,0.414,0.725,94,126,6,5,0,3,2
Brandon Nimmo,NYM,2016,21,OF,23,32,80,73,12,20,1,0,...,0.338,0.329,0.666,82,24,0,1,0,0,0
Gavin Cecchini,NYM,2016,24,SS,22,4,7,6,2,2,2,0,...,0.429,0.667,1.095,190,4,0,1,0,0,0


# SCRAPE TEAM NAMES

    > scrapes team name urls from table on main page of baseball reference
    > helps to concat url used in function below

In [4]:
base_url = 'http://www.baseball-reference.com/leagues/MLB/'
year = '/2016'
team_names = table_scraper.get_team_names(base_url, year)
team_names[0:10]

['/ARI',
 '/ATL',
 '/BAL',
 '/BOS',
 '/CHC',
 '/CHW',
 '/CIN',
 '/CLE',
 '/COL',
 '/DET']

# DOWNLOAD TEAM BATTING TABLES TO CSVS  

    > can download any teams/years combination for team batting tables

In [5]:
#create specific team-year combinations - cwd --5 minutes/year 
base_url = 'http://www.baseball-reference.com/teams'
years_lst = ['/2012', '/2013']
ext = '.shtml'

table_scraper.create_batting_links(base_url, team_names[0:5], years_lst, ext)

Saving in  os.getcwd() 

Saved csv for team_year - ARI_2012
Saved csv for team_year - ATL_2012
Saved csv for team_year - BAL_2012
Saved csv for team_year - BOS_2012
Saved csv for team_year - CHC_2012
Saved csv for team_year - ARI_2013
Saved csv for team_year - ATL_2013
Saved csv for team_year - BAL_2013
Saved csv for team_year - BOS_2013
Saved csv for team_year - CHC_2013


# CONCAT  ALL TEAM BATTING DFS 

    > file path must not contain sub-folders & must only have batting csvs
    > giant dataframe

In [6]:
team_path = ''
df1 = table_scraper.get_player_names(team_path, stats=True)
df1.shape

(8531, 30)

# GET PLAYER NAMES FROM CSVS TO SEARCH BR FOR SALARIES  
    > file path must not contain sub-folders & must only have batting csvs 
    > concatenates all csvs and extracts all player names from all csv files  

In [7]:
team_path = ''
player_names = table_scraper.get_player_names(team_path)
player_names[0:10]

['Chris Owings ',
 'Yasmany Tomas ',
 'Nick Ahmed ',
 'Rey Fuentes ',
 'Jeremy Hazelbaker ',
 'Jack Reinheimer ',
 'Ildemaro Vargas ',
 'Robbie Ray ',
 'Shelby Miller ',
 'Randall Delgado ']

# SALARY CRAWLER

    > given a list of player names from batting csvs ... 
       > ...iterate through hundreds of player name urls in br player index to scrape and save salary table to csv
           > ... takes 3-4 hours for all names in a given season

In [8]:
table_scraper.salary_crawl(player_names[0:5])

Saving in  os.getcwd() 

http://www.baseball-reference.com/players/a/ahmedni01.shtml
http://www.baseball-reference.com/players/f/fuentre01.shtml
'NoneType' object has no attribute 'find_all' ^^  -- salary table probably missing -- 
http://www.baseball-reference.com/players/h/hazelje01.shtml
http://www.baseball-reference.com/players/o/owingch01.shtml
http://www.baseball-reference.com/players/t/tomasya01.shtml


# MERGE PLAYER SALARY & TEAM BATTING DFS


    > file paths must not contain sub-folders & must only have csvs of that type
    > salaries dfs & team dfs indexed on team, name, year are merged and players w/o salaries are dropped

In [9]:
sal_path = ''
team_path = ''
merged_sal = table_scraper.merge_dfs(sal_path, team_path, 2016)

/Users/Anthony/Desktop/git_notebooks_clean/baseball_analysis.py:351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  sal_df1['YEARS'] = sal_df1['YEARS'].astype(int) # see pd_concat_flaot if doesnt work


In [10]:
table_scraper.sum_stat_by_team(team_path, '2016', '2B', get_cols=True)

Index(['NAME', 'TEAM_NAME', 'YEAR', 'RK', 'POS', 'AGE', 'G', 'PA', 'AB', 'R',
       'H', '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'BA', 'OBP',
       'SLG', 'OPS', 'OPS+', 'TB', 'GDP', 'HBP', 'SH', 'SF', 'IBB'],
      dtype='object')

# BAR GRAPH

    > plot any statistics that 'make sense' to be summed by team in a bar graph -- i.e. no percentages

In [11]:
# in 2017, houston is the team with the most home runs
table_scraper.sum_stat_by_team(team_path, 2017, 'HR')

# DIVIDE STAT BY STAT AND PLOT DISTRIBUTION AS HISTOGRAM

    > tuple[0] takes 1 column and divides it by another column
    > tuple[1] filters  tuple[0] column by "> tuple[1]" 
    > team path is path to location on computer where batting csvs are stored
    
    > in these 2 graphs... more AB before home runs in 2016 than in 2014

In [12]:
f1 = ('AB', 20)
s1 = ('HR', 5)
team_path = ''
table_scraper.stat_divided_by_stat(team_path, f1,s1, 2014, bins_int=20)

/Users/Anthony/Desktop/git_notebooks_clean/baseball_analysis.py:519: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  years_df1[new_col_name] = years_df1[first_stat_tup[0]] / years_df1[second_stat_tup[0]]


In [13]:
f1 = ('AB', 20)
s1 = ('HR', 5)
team_path = '/Users/Anthony/Desktop/git_notebooks_clean/team_batting_2012-2017'
table_scraper.stat_divided_by_stat(team_path, f1,s1, 2016, bins_int=20)

/Users/Anthony/Desktop/git_notebooks_clean/baseball_analysis.py:519: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  years_df1[new_col_name] = years_df1[first_stat_tup[0]] / years_df1[second_stat_tup[0]]


# SCATTERPLOT   
    > get column names 
    > salaries dfs & team dfs indexed on team, name, year are merged and players w/o salaries are dropped
    > the code below plots players with salaries above a number but below another numner (above $1,000,000 but      below $10,000,000)
    > had to play greater than 20 games

In [14]:
sal_path = ''
team_path = ''
merged_sal = table_scraper.merge_dfs(sal_path, team_path, 2017)

table_scraper.scatterplot_salary(merged_sal,1000000, 10000000, ('G', 20), get_plot_df=True).columns

/Users/Anthony/Desktop/git_notebooks_clean/baseball_analysis.py:351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  sal_df1['YEARS'] = sal_df1['YEARS'].astype(int) # see pd_concat_flaot if doesnt work


Index(['NAME', 'TEAM_NAME', 'YEARS', 'RK', 'POS', 'AGE', 'G', 'PA', 'AB', 'R',
       'H', '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'BA', 'OBP',
       'SLG', 'OPS', 'OPS+', 'TB', 'GDP', 'HBP', 'SH', 'SF', 'IBB', 'SALARY',
       'POSITION', 'HEIGHT', 'WEIGHT', 'PLOT_SALARY'],
      dtype='object')

In [15]:
table_scraper.scatterplot_salary(merged_sal,10000000, 20000000, ('G', 20))
#table_scraper.scatterplot_salary(merged_sal,1000000, 10000000, ('G', 20), get_plot_df=True)

/Applications/anaconda/lib/python3.6/site-packages/bokeh/models/sources.py:81: BokehUserWarning: ColumnDataSource's columns must be of the same length
  lambda: warnings.warn("ColumnDataSource's columns must be of the same length", BokehUserWarning))
/Applications/anaconda/lib/python3.6/site-packages/bokeh/models/sources.py:81: BokehUserWarning: ColumnDataSource's columns must be of the same length
  lambda: warnings.warn("ColumnDataSource's columns must be of the same length", BokehUserWarning))
/Applications/anaconda/lib/python3.6/site-packages/bokeh/models/sources.py:81: BokehUserWarning: ColumnDataSource's columns must be of the same length
  lambda: warnings.warn("ColumnDataSource's columns must be of the same length", BokehUserWarning))
/Applications/anaconda/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: Setting Plot property 'title' using a string was deprecated in 0.12.0,
            and will be removed. The title is now an object on Plot (w